# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

In [ ]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
from pathlib import Path

# Load model
transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rewards = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

# Fixed policy: always do-nothing
A = "do-nothing"
T = transitions[transitions["action"] == A].copy()
R = rewards[rewards["action"] == A].set_index("state")["reward"].to_dict()

# State set
states = sorted(set(T["state"]).union(T["next_state"]))
idx = {s: i for i, s in enumerate(states)}
n = len(states)

# Transition matrix P under the fixed policy (rows: s, cols: s')
P = np.zeros((n, n))
for _, row in T.iterrows():
    P[idx[row["state"]], idx[row["next_state"]]] += row["probability"]

# Per-step reward vector r(s, A). Rewards are negative discomfort.
r = np.array([R.get(s, 0.0) for s in states])

# Policy evaluation with gamma = 1 (episodic; recovered is absorbing with 0 reward)
# Solve (I - P) V = r; if singular due to numerics, fall back to iterative evaluation
I = np.eye(n)
try:
    V = np.linalg.solve(I - P, r)
except np.linalg.LinAlgError:
    V = np.zeros(n)
    for _ in range(100000):
        V_new = r + P @ V
        if np.max(np.abs(V_new - V)) < 1e-12:
            V = V_new
            break
        V = V_new

# Convert expected cumulative rewards to expected discomfort (positive)
expected_discomfort = -V

do_nothing_df = pd.DataFrame(
    {"state": states, "expected_discomfort": expected_discomfort}
)
do_nothing_df

,state,expected_discomfort
0,exposed-1,3.413333
1,exposed-2,4.266667
2,exposed-3,5.333333
3,recovered,-0.000000
4,symptoms-1,6.666667
5,symptoms-2,5.000000
6,symptoms-3,1.666667


Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

In [2]:
# YOUR CHANGES HERE

out_path = Path("do-nothing-discomfort.tsv")
do_nothing_df.to_csv(out_path, sep="\t", index=False)
print(f"Saved: {out_path.resolve()}")

Saved: /workspaces/dx704-project-06/do-nothing-discomfort.tsv


Submit "do-nothing-discomfort.tsv" in Gradescope.

## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

In [3]:
# YOUR CHANGES HERE

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rewards = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

# Build MDP components
actions = sorted(transitions["action"].unique())
states = sorted(set(transitions["state"]).union(transitions["next_state"]).union(rewards["state"]))
idx = {s: i for i, s in enumerate(states)}
n = len(states)

# Transition matrices P[a] and reward vectors r[a]
P = {a: np.zeros((n, n)) for a in actions}
for _, row in transitions.iterrows():
    P[row["action"]][idx[row["state"]], idx[row["next_state"]]] += row["probability"]

R = {a: np.full(n, 0.0) for a in actions}
for _, row in rewards.iterrows():
    R[row["action"]][idx[row["state"]]] = row["reward"]

# Value Iteration (gamma = 1 for episodic; recovered should be absorbing)
gamma = 1.0
V = np.zeros(n)
max_iters, tol = 200000, 1e-12

for _ in range(max_iters):
    Qs = [R[a] + gamma * (P[a] @ V) for a in actions]
    V_new = np.max(Qs, axis=0)
    if np.max(np.abs(V_new - V)) < tol:
        V = V_new
        break
    V = V_new
else:
    # If gamma=1 doesn't converge (rare), slightly discount to guarantee convergence and retry
    gamma = 0.9999
    V = np.zeros(n)
    for _ in range(max_iters):
        Qs = [R[a] + gamma * (P[a] @ V) for a in actions]
        V_new = np.max(Qs, axis=0)
        if np.max(np.abs(V_new - V)) < tol:
            V = V_new
            break
        V = V_new

# Greedy optimal action for each state (ties broken by action order)
Q_final = {a: R[a] + gamma * (P[a] @ V) for a in actions}
best_action_idx = np.argmax(np.vstack([Q_final[a] for a in actions]), axis=0)
optimal_actions = [actions[i] for i in best_action_idx]

# Save results for future use
opt_actions_df = pd.DataFrame({"state": states, "action": optimal_actions})
opt_V = pd.Series(V, index=states) 
opt_actions_df

,state,action
0,exposed-1,sleep-8
1,exposed-2,sleep-8
2,exposed-3,sleep-8
3,recovered,do-nothing
4,symptoms-1,drink-oj
5,symptoms-2,drink-oj
6,symptoms-3,drink-oj


Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

In [4]:
# YOUR CHANGES HERE

out_path = "minimum-discomfort-actions.tsv"
opt_actions_df.to_csv(out_path, sep="\t", index=False)
print(f"Saved: {out_path}")

Saved: minimum-discomfort-actions.tsv


Submit "minimum-discomfort-actions.tsv" in Gradescope.

## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

In [5]:
# YOUR CHANGES HERE

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rewards = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")
opt_actions = pd.read_csv("minimum-discomfort-actions.tsv", sep="\t")

# Consistent state ordering
states = sorted(set(transitions["state"]).union(transitions["next_state"]).union(rewards["state"]).union(opt_actions["state"]))
idx = {s: i for i, s in enumerate(states)}
n = len(states)

# Deterministic policy pi*(s)
pi = dict(zip(opt_actions["state"], opt_actions["action"]))

# Build P*pi and r*pi
P_pi = np.zeros((n, n))
r_pi = np.zeros(n)
rew_lookup = rewards.set_index(["state", "action"])["reward"].to_dict()

for _, row in transitions.iterrows():
    s = row["state"]; a = pi.get(s)
    if a is None or row["action"] != a:
        continue
    i, j = idx[s], idx[row["next_state"]]
    P_pi[i, j] += row["probability"]

for s in states:
    a = pi.get(s)
    r_pi[idx[s]] = rew_lookup.get((s, a), 0.0)

# Policy evaluation with gamma=1 (episodic): (I - P*pi) V = r*pi
I = np.eye(n)
try:
    V = np.linalg.solve(I - P_pi, r_pi)
except np.linalg.LinAlgError:
    V = np.zeros(n)
    for _ in range(200000):
        V_new = r_pi + P_pi @ V
        if np.max(np.abs(V_new - V)) < 1e-12:
            V = V_new
            break
        V = V_new

# Expected discomfort (positive) is the negative of expected cumulative reward
expected_discomfort = -V

minimum_discomfort_df = pd.DataFrame({"state": states, "expected_discomfort": expected_discomfort})
minimum_discomfort_df

,state,expected_discomfort
0,exposed-1,0.75
1,exposed-2,1.50
2,exposed-3,3.00
3,recovered,-0.00
4,symptoms-1,6.00
5,symptoms-2,4.50
6,symptoms-3,1.50


Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

In [6]:
# YOUR CHANGES HERE

out_path = "minimum-discomfort-values.tsv"
minimum_discomfort_df.to_csv(out_path, sep="\t", index=False)
print(f"Saved: {out_path}")

Saved: minimum-discomfort-values.tsv


Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick and 0 if the current state corresponds to being better.
To be clear, the action does not matter for this reward function.


In [ ]:
# YOUR CHANGES HERE

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rewards_orig = pd.read_csv("twizzleflu-rewards.tsv", sep="\t") 

actions = sorted(transitions["action"].unique())
states = sorted(set(transitions["state"]).union(transitions["next_state"]).union(rewards_orig["state"]))

# Identify "better" (recovered) states:
# 1) prefer name keywords; 2) fallback to absorbing states
kw = {"recover", "better", "healthy"}
better_states = {s for s in states if any(k in s.lower() for k in kw)}

if not better_states:
    out_by_pair = transitions.groupby(["state","next_state"])["probability"].sum().reset_index()
    row_sum = transitions.groupby("state")["probability"].sum()
    absorbing = set()
    for s in states:
        if row_sum.get(s, 0.0) == 0.0:
            continue
        p_self = float(out_by_pair[(out_by_pair["state"]==s) & (out_by_pair["next_state"]==s)]["probability"].sum())
        if abs(p_self - 1.0) < 1e-9:
            absorbing.add(s)
    better_states = absorbing

# Build the new reward function: -1 if sick, 0 if better; independent of action
duration_rewards = pd.DataFrame(
    [{"state": s, "action": a, "reward": (0.0 if s in better_states else -1.0)}
     for s in states for a in actions]
)

Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [ ]:
# YOUR CHANGES HERE

duration_rewards.to_csv("duration-rewards.tsv", sep="\t", index=False)
print("Saved: duration-rewards.tsv")

Saved: duration-rewards.tsv


Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

In [13]:
# YOUR CHANGES HERE

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
duration_rewards = pd.read_csv("duration-rewards.tsv", sep="\t")

actions = sorted(transitions["action"].unique())
states = sorted(set(transitions["state"]).union(transitions["next_state"]))
idx = {s: i for i, s in enumerate(states)}
n = len(states)

# Transition matrices per action
P = {a: np.zeros((n, n)) for a in actions}
for _, row in transitions.iterrows():
    P[row["action"]][idx[row["state"]], idx[row["next_state"]]] += row["probability"]

# Reward vectors per action (action-independent here, but we keep the structure)
R = {
    a: duration_rewards[duration_rewards["action"] == a]
         .set_index("state")["reward"]
         .reindex(states).to_numpy()
    for a in actions
}

# Value Iteration (gamma=1; episodic). Fallback to slight discount if needed for convergence.
gamma = 1.0
V = np.zeros(n)
tol, max_iters = 1e-12, 200000

for _ in range(max_iters):
    Qs = [R[a] + gamma*(P[a] @ V) for a in actions]
    V_new = np.max(Qs, axis=0)
    if np.max(np.abs(V_new - V)) < tol:
        V = V_new
        break
    V = V_new
else:
    gamma = 0.9999
    V = np.zeros(n)
    for _ in range(max_iters):
        Qs = [R[a] + gamma*(P[a] @ V) for a in actions]
        V_new = np.max(Qs, axis=0)
        if np.max(np.abs(V_new - V)) < tol:
            V = V_new
            break
        V = V_new

# Greedy optimal action for each state (ties by action sort order)
Q_final = {a: R[a] + gamma*(P[a] @ V) for a in actions}
best_action_idx = np.argmax(np.vstack([Q_final[a] for a in actions]), axis=0)
optimal_actions = [actions[i] for i in best_action_idx]

Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.

In [14]:
# YOUR CHANGES HERE

out_df = pd.DataFrame({"state": states, "action": optimal_actions})
out_df.to_csv("minimum-duration-actions.tsv", sep="\t", index=False)
print("Saved: minimum-duration-actions.tsv")

Saved: minimum-duration-actions.tsv


Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

In [ ]:
# YOUR CHANGES HERE

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")


actions = sorted(transitions["action"].unique())
states = sorted(set(transitions["state"]).union(transitions["next_state"]))

# Identify "better" (absorbing/healthy) states by keyword then fallback to absorbing
kw = {"recover", "better", "healthy"}
better_states = [s for s in states if any(k in s.lower() for k in kw)]
if not better_states:
    better_states = []
    for s in states:
        all_self = True
        for a in actions:
            sub = transitions[(transitions["state"] == s) & (transitions["action"] == a)]
            if sub.empty:
                continue
            p_self = sub[sub["next_state"] == s]["probability"].sum()
            if abs(p_self - 1.0) > 1e-12:
                all_self = False
                break
        if all_self:
            better_states.append(s)

# Build P(s,a,.)
state_index = {s: i for i, s in enumerate(states)}
nS = len(states)
P = {a: np.zeros((nS, nS)) for a in actions}
for a in actions:
    sub = transitions[transitions["action"] == a]
    for _, row in sub.iterrows():
        i = state_index[row["state"]]
        j = state_index[row["next_state"]]
        P[a][i, j] += float(row["probability"])

# Value iteration for undiscounted stochastic shortest path:
# cost per step is 1 in non-better states, 0 in better states; goal is to minimize expected steps.
cost = np.ones(nS)
abs_idx = set(state_index[s] for s in better_states)
for i in abs_idx:
    cost[i] = 0.0

V = np.zeros(nS)
max_iters = 10000
tol = 1e-12

for _ in range(max_iters):
    V_prev = V.copy()
    Qs = [P[a] @ V_prev for a in actions]
    stacked = np.vstack(Qs)  # shape (nA, nS)
    V = cost + stacked.min(axis=0)
    # Keep absorbing states pinned to 0
    if abs_idx:
        for i in abs_idx:
            V[i] = 0.0
    if np.max(np.abs(V - V_prev)) < tol:
        break

expected_sick_days_df = pd.DataFrame({
    "state": states,
    "expected_sick_days": V
})
expected_sick_days_df

,state,expected_sick_days
0,exposed-1,3.000000
1,exposed-2,4.000000
2,exposed-3,6.000000
3,recovered,0.000000
4,symptoms-1,10.000000
5,symptoms-2,6.666667
6,symptoms-3,3.333333


In [21]:
transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
policy_df = pd.read_csv("minimum-duration-actions.tsv", sep="\t")

actions = sorted(transitions["action"].unique())
states = sorted(set(transitions["state"]).union(transitions["next_state"]))
state_index = {s: i for i, s in enumerate(states)}
n = len(states)

kw = {"recover", "better", "healthy"}
better_states = [s for s in states if any(k in s.lower() for k in kw)]

# Build policy-specific transition matrix
P_pi = np.zeros((n, n))
policy_map = dict(zip(policy_df["state"], policy_df["action"]))
for s in states:
    i = state_index[s]
    a = policy_map[s]
    sub = transitions[(transitions["state"] == s) & (transitions["action"] == a)]
    if sub.empty:
        P_pi[i, i] = 1.0
    else:
        for _, row in sub.iterrows():
            P_pi[i, state_index[row["next_state"]]] += float(row["probability"])

absorbing = set(better_states)
transient_states = [s for s in states if s not in absorbing]
idx_transient = [state_index[s] for s in transient_states]

expected = np.zeros(n)
if idx_transient:
    Q = P_pi[np.ix_(idx_transient, idx_transient)]
    A = np.eye(Q.shape[0]) - Q
    ones = np.ones(Q.shape[0])
    try:
        t_trans = np.linalg.solve(A, ones)
    except np.linalg.LinAlgError:
        t_trans, *_ = np.linalg.lstsq(A, ones, rcond=None)
    for k, s in enumerate(transient_states):
        expected[state_index[s]] = t_trans[k]

for s in absorbing:
    expected[state_index[s]] = 0.0

expected_sick_days_df = pd.DataFrame({"state": states, "expected_sick_days": expected})
expected_sick_days_df

,state,expected_sick_days
0,exposed-1,3.000000
1,exposed-2,4.000000
2,exposed-3,6.000000
3,recovered,0.000000
4,symptoms-1,10.000000
5,symptoms-2,6.666667
6,symptoms-3,3.333333


Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [22]:
# YOUR CHANGES HERE

out_path = "minimum-duration-days.tsv"
expected_sick_days_df.to_csv(out_path, sep="\t", index=False)
print(f"Saved: {out_path}")

Saved: minimum-duration-days.tsv


Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

In [24]:
# YOUR CHANGES HERE

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rewards_df = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")
speed_policy_df = pd.read_csv("minimum-duration-actions.tsv", sep="\t")

actions = sorted(transitions["action"].unique())
states = sorted(set(transitions["state"]).union(transitions["next_state"]).union(rewards_df["state"]))
state_index = {s: i for i, s in enumerate(states)}
nS = len(states)

kw = {"recover", "better", "healthy"}
better_states = [s for s in states if any(k in s.lower() for k in kw)]
absorbing_idx = {state_index[s] for s in better_states}

# P and c
P = {a: np.zeros((nS, nS)) for a in actions}
for a in actions:
    sub = transitions[transitions["action"] == a]
    for _, row in sub.iterrows():
        P[a][state_index[row["state"]], state_index[row["next_state"]]] += float(row["probability"])

c = {a: np.zeros(nS) for a in actions}
for _, row in rewards_df.iterrows():
    c[row["action"]][state_index[row["state"]]] = float(-row["reward"])
for a in actions:
    for i in absorbing_idx:
        c[a][i] = 0.0

# Speed policy eval
speed_policy = dict(zip(speed_policy_df["state"], speed_policy_df["action"]))
P_pi = np.zeros((nS, nS)); c_pi = np.zeros(nS)
for s in states:
    i = state_index[s]
    a = speed_policy[s]
    P_pi[i, :] = P[a][i, :]
    c_pi[i] = c[a][i]

transient_states = [s for s in states if state_index[s] not in absorbing_idx]
idx_T = [state_index[s] for s in transient_states]
speed_V = np.zeros(nS)
if idx_T:
    Q = P_pi[np.ix_(idx_T, idx_T)]
    A = np.eye(Q.shape[0]) - Q
    b = c_pi[idx_T]
    try:
        v_T = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        v_T, *_ = np.linalg.lstsq(A, b, rcond=None)
    for k, s in enumerate(transient_states):
        speed_V[state_index[s]] = v_T[k]

# Optimal discomfort (value iteration)
opt_V = np.zeros(nS)
max_iters = 10000; tol = 1e-12
for _ in range(max_iters):
    prev = opt_V.copy()
    stacked = np.vstack([c[a] + P[a] @ prev for a in actions])
    opt_V = stacked.min(axis=0)
    if absorbing_idx:
        for i in absorbing_idx:
            opt_V[i] = 0.0
    if np.max(np.abs(opt_V - prev)) < tol:
        break

policy_comparison_df = pd.DataFrame({
    "state": states,
    "speed_discomfort": speed_V,
    "minimize_discomfort": opt_V
})
policy_comparison_df

,state,speed_discomfort,minimize_discomfort
0,exposed-1,0.833333,0.75
1,exposed-2,1.666667,1.50
2,exposed-3,3.333333,3.00
3,recovered,0.000000,0.00
4,symptoms-1,6.666667,6.00
5,symptoms-2,5.000000,4.50
6,symptoms-3,1.666667,1.50


Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [26]:
# YOUR CHANGES HERE

policy_comparison_df.to_csv("policy-comparison.tsv", sep="\t", index=False)
print("Saved:", "/mnt/data/policy-comparison.tsv")
print(policy_comparison_df.to_string(index=False))

Saved: /mnt/data/policy-comparison.tsv
     state  speed_discomfort  minimize_discomfort
 exposed-1          0.833333                 0.75
 exposed-2          1.666667                 1.50
 exposed-3          3.333333                 3.00
 recovered          0.000000                 0.00
symptoms-1          6.666667                 6.00
symptoms-2          5.000000                 4.50
symptoms-3          1.666667                 1.50


Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.